In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.sql("select 'spark' as hello")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [131]:
df = spark.read.csv("toy_dataset.csv")
df.show()

+------+------+------+---+-------+-------+
|   _c0|   _c1|   _c2|_c3|    _c4|    _c5|
+------+------+------+---+-------+-------+
|Number|  City|Gender|Age| Income|Illness|
|     1|Dallas|  Male| 41|40367.0|     No|
|     2|Dallas|  Male| 54|45084.0|     No|
|     3|Dallas|  Male| 42|52483.0|     No|
|     4|Dallas|  Male| 40|40941.0|     No|
|     5|Dallas|  Male| 46|50289.0|     No|
|     6|Dallas|Female| 36|50786.0|     No|
|     7|Dallas|Female| 32|33155.0|     No|
|     8|Dallas|  Male| 39|30914.0|     No|
|     9|Dallas|  Male| 51|68667.0|     No|
|    10|Dallas|Female| 30|50082.0|     No|
|    11|Dallas|Female| 48|41524.0|    Yes|
|    12|Dallas|  Male| 47|54777.0|     No|
|    13|Dallas|  Male| 46|62749.0|     No|
|    14|Dallas|Female| 42|50894.0|     No|
|    15|Dallas|Female| 61|38429.0|     No|
|    16|Dallas|  Male| 43|34074.0|     No|
|    17|Dallas|  Male| 27|50398.0|     No|
|    18|Dallas|  Male| 38|46373.0|    Yes|
|    19|Dallas|  Male| 47|51137.0|     No|
+------+---

In [132]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)



In [133]:
df = spark.read.csv("toy_dataset.csv", inferSchema = True, header = True)
df.printSchema()

root
 |-- Number: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Income: double (nullable = true)
 |-- Illness: string (nullable = true)



In [11]:
df.count()

150000

In [12]:
df.where(df.Gender == "Male").count()

83800

In [130]:
df2 = df.sample(0.01)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [14]:
df.groupBy("City").count().show()

+---------------+-----+
|           City|count|
+---------------+-----+
|         Dallas|19707|
|    Los Angeles|32173|
|      San Diego| 4881|
|         Austin|12292|
|  New York City|50307|
|Washington D.C.| 8120|
|  Mountain View|14219|
|         Boston| 8301|
+---------------+-----+



In [15]:
df.groupBy("City").count().count()

8

In [22]:
print("Ill: ", df.where(df.Illness == "Yes").count())
print("Not Ill: ", df.where(df.Illness == "No").count())

Ill:  12139
Not Ill:  137861


In [24]:
df2 = df.groupBy("Illness").count().show()

+-------+------+
|Illness| count|
+-------+------+
|     No|137861|
|    Yes| 12139|
+-------+------+



In [35]:
df2 = df.groupBy("Illness").count()
rows = df2.select("count").collect()

print(rows[0][0])
print(type(rows[0]))

137861
<class 'pyspark.sql.types.Row'>


In [58]:
male_num = df.where(df.Gender == "Male").count()
female_num = df.where(df.Gender == "Female").count()
total = male_num + female_num
print(total)
male_percent = male_num/total*100
female_percent = female_num/total*100
print("Male ", male_percent, "+ Female ", female_percent, " = ", male_percent + female_percent)
print("Total: {} + {} = {}".format(male_num, female_num, total))

150000
Male  55.86666666666667 + Female  44.13333333333333  =  100.0
Total: 83800 + 66200 = 150000


In [62]:
df2 = df.where(df.Illness=="Yes").groupBy(df.Gender).count()
df2.show()

+------+-----+
|Gender|count|
+------+-----+
|Female| 5331|
|  Male| 6808|
+------+-----+



In [69]:
from pyspark.sql.functions import col, asc,desc
df2 = df.where(df.Illness=="Yes").groupBy(df.City).count().sort(col("count").desc())
df2.show()
df2.take(1)

+---------------+-----+
|           City|count|
+---------------+-----+
|  New York City| 4021|
|    Los Angeles| 2568|
|         Dallas| 1613|
|  Mountain View| 1178|
|         Austin| 1011|
|         Boston|  686|
|Washington D.C.|  668|
|      San Diego|  394|
+---------------+-----+



[Row(City='New York City', count=4021)]

In [71]:
from pyspark.sql.functions import mean

In [73]:
df2 = df.where(df.Illness=="Yes").select(mean("Income")).show()

+----------------+
|     avg(Income)|
+----------------+
|91277.8753604086|
+----------------+



In [75]:
df2 = df.where(df.Illness=="No").select(mean("Income")).withColumnRenamed("avg(Income)", "Mean Healthy Income").show()

+-------------------+
|Mean Healthy Income|
+-------------------+
|  91250.59017416093|
+-------------------+



In [81]:
avgIll = df.where(df.Illness=="Yes").select(mean("Income")).withColumnRenamed("avg(Income)", "Mean Ill Income")
avgHealthy = df.where(df.Illness=="No").select(mean("Income")).withColumnRenamed("avg(Income)", "Mean Healthy Income")
avgIll.join(avgHealthy).show(vertical=True)

-RECORD 0--------------------------------
 Mean Ill Income     | 91277.8753604086  
 Mean Healthy Income | 91250.59017416093 



In [84]:
ageIll = df.where(df.Illness=="Yes").select(mean("Age")).withColumnRenamed("avg(Age)", "Average Age of Ill People").show()

+-------------------------+
|Average Age of Ill People|
+-------------------------+
|        45.02084191449048|
+-------------------------+



In [87]:
ageIll = df.where(df.Illness=="Yes").select("Age").withColumnRenamed("avg(Age)", "Average Age of Ill People").summary().show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|             12139|
|   mean| 45.02084191449048|
| stddev|11.554129904969162|
|    min|                25|
|    25%|                35|
|    50%|                45|
|    75%|                55|
|    max|                65|
+-------+------------------+



In [118]:
ill = df.where(df.Illness=="No").select("City", "Age").sort(col("Age").desc()).collect()
print(ill[0])

Row(City='Dallas', Age=65)


In [124]:
from pyspark.sql.functions import max
df.select(max("Age")).show()

+--------+
|max(Age)|
+--------+
|      65|
+--------+



In [138]:
df3 = spark.read.csv("Subsidized_Housing.csv", inferSchema = True, header = True)
df3.printSchema()

root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- OBJECTID: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- NAME: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADMIN: string (nullable = true)
 |-- FIRST_BUIL: integer (nullable = true)
 |-- LAST_BUILT: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MSHDA: integer (nullable = true)
 |-- LIHTC: integer (nullable = true)
 |-- MARKET: integer (nullable = true)
 |-- RAP: integer (nullable = true)
 |-- SEC_515_UN: integer (nullable = true)
 |-- SEC_202_UN: integer (nullable = true)
 |-- BARRIER_FR: integer (nullable = true)
 |-- REHAB_CHOI: integer (nullable = true)
 |-- PUBLIC_HSG: integer (nullable = true)
 |-- SEC_221D3_: integer (nullable = true)
 |-- SEC_236_UN: integer (

In [148]:
df3.select("Name", "Address", "City", "State", "Zip", "County", "Phone", "Admin", "FIRST_BUIL", "LAST_BUILT", "Age").show(vertical=True)

-RECORD 0--------------------------
 Name       | Orion Four Seasons   
 Address    | Baldwin Road         
 City       | Orion Twp            
 State      | MI                   
 Zip        | 0                    
 County     | Oakland              
 Phone      |                      
 Admin      | MSHDA                
 FIRST_BUIL | 2002                 
 LAST_BUILT | 2002                 
 Age        | 9                    
-RECORD 1--------------------------
 Name       | Island Woods         
 Address    | Gray's Drive         
 City       | Grosse Ile           
 State      | MI                   
 Zip        | 48138                
 County     | Wayne                
 Phone      | (734)476-8029        
 Admin      | MSHDA                
 FIRST_BUIL | 2002                 
 LAST_BUILT | 2002                 
 Age        | 9                    
-RECORD 2--------------------------
 Name       | Flint Hsg Commission 
 Address    | Scattered Sites      
 City       | Flint         

In [155]:
df3.select("Name", "Age").sort(col("Age").desc()).show()

+--------------------+---+
|                Name|Age|
+--------------------+---+
|          Field Apts| 88|
|      Lee Crest Apts| 81|
|   Seventy West Apts| 79|
|Parkside Homes An...| 73|
|   Colonel Hamtramck| 70|
|     Sojourner Truth| 69|
|        Ryan Ct Apts| 66|
|    West Boston Apts| 61|
|   Northlawn Gardens| 56|
|     Voisine Terrace| 53|
|       Wayne Housing| 52|
|    Le Moyng Gardens| 52|
|         Sky Harbour| 51|
|     Rochester Manor| 51|
|    Parklane Th Coop| 51|
|  Clemens Homes 28-1| 50|
|    Broadway Terrace| 49|
|      Rochdale Court| 49|
|       Lurie Terrace| 49|
|    Wade Mcree Plaza| 49|
+--------------------+---+
only showing top 20 rows



In [171]:
df4 = spark.read.json("1880-2016.json")
df4.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- 1880: string (nullable = true)
 |    |-- 1881: string (nullable = true)
 |    |-- 1882: string (nullable = true)
 |    |-- 1883: string (nullable = true)
 |    |-- 1884: string (nullable = true)
 |    |-- 1885: string (nullable = true)
 |    |-- 1886: string (nullable = true)
 |    |-- 1887: string (nullable = true)
 |    |-- 1888: string (nullable = true)
 |    |-- 1889: string (nullable = true)
 |    |-- 1890: string (nullable = true)
 |    |-- 1891: string (nullable = true)
 |    |-- 1892: string (nullable = true)
 |    |-- 1893: string (nullable = true)
 |    |-- 1894: string (nullable = true)
 |    |-- 1895: string (nullable = true)
 |    |-- 1896: string (nullable = true)
 |    |-- 1897: string (nullable = true)
 |    |-- 1898: string (nullable = true)
 |    |-- 1899: string (nullable = true)
 |    |-- 1900: string (nullable = true)
 |    |-- 1901: string (nullable = true)
 |    |-- 1902: string (nullable = true)
 |    |-- 1903: 

In [170]:
df4.select("data", "description").show(vertical=True)

-RECORD 0---------------------------
 data        | {-0.12, -0.09, -0... 
 description | {1901-2000, -999,... 

